In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131183 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.13-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.13-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.13-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [0]:
# 指定Google Drive云端硬盘的根目录，名为drive
!mkdir -p drive
!google-drive-ocamlfuse drive

In [3]:
cd drive

/content/drive


In [0]:
# 指定當前的工作資料夾
import os
# 此處為google drive中的檔案路徑,drive為之前指定的工作根目錄，要加上
os.chdir("Colab Notebooks/玉山秋季賽") 

In [0]:
os.chdir("0930") 

In [0]:
os.chdir("洗完資料") 

In [7]:
ls

submission_test.csv  test_0920.csv  train_0920.csv


In [0]:
import lightgbm as lgb
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn import datasets

In [0]:
#匯入訓練資料
#雲端匯入
df = pd.read_csv('./train_0920.csv', encoding='utf-8')

In [10]:
data = df.drop(["fraud_ind"], axis=1) #剔除 y
X = data 
y = df.fraud_ind #選擇 y
y.dtypes

dtype('int64')

In [0]:
#切割訓練70%跟測試30%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01)

In [0]:
model = lgb.LGBMClassifier(
                task = 'train',
                application = 'binary',
                boosting = 'goss',
                num_iterations = 60000,
                learning_rate = 0.001,
                num_leaves = 50,
                tree_learner = 'serial',
                num_threads = 3,
                max_depth = 8,
                min_data_in_leaf = 50,
                feature_fraction = 1,
                bagging_fraction = 1,
                bagging_freq = 10,
                lambda_l2 = 0.0001
)
model.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


In [0]:
#匯入玉山測試集資料
#uploaded = files.upload()
#雲端匯入
#dftest = pd.read_csv(io.BytesIO(uploaded['test_1w_0607.csv']))
dftest = pd.read_csv('./test_0920.csv', encoding='utf-8')

In [0]:
data_test = dftest[X_test.columns]
data_test.shape[0]

421665

In [0]:
#做測試集預測
y_hat_test = model.predict(data_test)
pdata=pd.DataFrame(y_hat_test)
test_yhat = pdata.rename(columns={0: 'fraud_ind'})
print(test_yhat.shape[0])

421665


In [0]:
#匯入訓練資料
#uploaded = files.upload()
#雲端匯入
summit = pd.read_csv('./submission_test.csv', encoding='utf-8')
summit = summit.drop(["fraud_ind"], axis=1) #剔除 old total.price

In [0]:
summit_new = pd.concat([summit, test_yhat],axis=1) 
summit_new .iloc[:5,:3]

,txkey,fraud_ind
0,592489,0
1,592452,0
2,590212,0
3,590209,0
4,592488,0


#輸出

#先退回目錄在指進其他資料夾

In [0]:
cd ..

/content/drive/Colab Notebooks/玉山秋季賽/0930


In [0]:
os.chdir("結果")

In [0]:
summit_new.to_csv('0930A.csv', sep=',', encoding='utf-8',index=0) #不保存行索引